In [1]:
import pandas as pd
import pyarrow as pa

In [2]:
s = pd.Series(
    [1.234, 5.6789],
)

s

0    1.2340
1    5.6789
dtype: float64

In [3]:
PRECISION = 38
SCALE = 3

In [4]:
print(f"{s.memory_usage(deep=True) = }")

s.memory_usage(deep=True) = 144


In [5]:
# Method 1: Convert the `pd.Series` with a `.dtype` of `float64`
#           to a `pd.Series` holding roundings of the same floating-point numbers
#           (with those roundings performed according to
#           the values of `PRECISION` and `SCALE`)

m_1_pa_array_rounded: pa.Array = pa.Array.from_pandas(s)
print(f"{m_1_pa_array_rounded.to_pandas().astype('float64').memory_usage(deep=True) = }")

m_1_pa_array_rounded_cast = m_1_pa_array_rounded.cast(
    target_type=pa.decimal128(PRECISION, scale=SCALE)
)
print(f"{m_1_pa_array_rounded_cast.to_pandas().astype('float64').memory_usage(deep=True) = }")

m_1_new_s = m_1_pa_array_rounded_cast.to_pandas()
print(f"{m_1_new_s.memory_usage(deep=True) = }")

m_1_pa_array_rounded.to_pandas().astype('float64').memory_usage(deep=True) = 144
m_1_pa_array_rounded_cast.to_pandas().astype('float64').memory_usage(deep=True) = 144
m_1_new_s.memory_usage(deep=True) = 352


In [6]:
m_1_new_s

0    1.234
1    5.679
dtype: object

In [7]:
# Method 2: Convert the `pd.Series` with a `.dtype` of `float64`
#           to a `pd.Series` holding roundings of the same floating-point numbers
#           (with those roundings performed according to
#           the values of `PRECISION` and `SCALE`)

m_2_pa_array_rounded: pa.Array = pa.Array.from_pandas(s)
print(f"{m_2_pa_array_rounded.to_pandas().astype('float64').memory_usage(deep=True) = }")

m_2_pa_array_rounded_cast = m_2_pa_array_rounded.cast(
    pa.decimal128(PRECISION, scale=SCALE)
)
print(f"{m_2_pa_array_rounded_cast.to_pandas().astype('float64').memory_usage(deep=True) = }")

m_2_new_s = m_2_pa_array_rounded_cast.to_pandas(
    types_mapper=pd.ArrowDtype
)
print(f"{m_2_new_s.memory_usage(deep=True) = }")

m_2_pa_array_rounded.to_pandas().astype('float64').memory_usage(deep=True) = 144
m_2_pa_array_rounded_cast.to_pandas().astype('float64').memory_usage(deep=True) = 144
m_2_new_s.memory_usage(deep=True) = 160


In [8]:
m_2_new_s

0    1.234
1    5.679
dtype: decimal128(38, 3)[pyarrow]

In [9]:
isinstance(m_2_new_s.dtype, pd.ArrowDtype)

True

In [10]:
[field for field in dir(m_2_new_s.dtype) if not field.startswith('__')]

['_cache',
 '_can_hold_na',
 '_get_common_dtype',
 '_is_boolean',
 '_is_numeric',
 '_metadata',
 '_parse_temporal_dtype_string',
 'construct_array_type',
 'construct_from_string',
 'empty',
 'is_dtype',
 'itemsize',
 'kind',
 'na_value',
 'name',
 'names',
 'numpy_dtype',
 'pyarrow_dtype',
 'storage',
 'type']

In [11]:
m_2_new_s.dtype.pyarrow_dtype

Decimal128Type(decimal128(38, 3))

In [12]:
isinstance(m_2_new_s.dtype.pyarrow_dtype, pa.Decimal128Type)

True